<a href="https://colab.research.google.com/github/Brandonj171/Brandonj171/blob/main/FACIALRECOGNITION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
print("CUDA Device Name:", torch.cuda.get_device_name(0))
import zipfile
import os

zip_path = "/content/drive/MyDrive/known (3).zip"
extract_folder = "/content/known"

os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)


print("Extracted files:", os.listdir(extract_folder))

zip_path = "/content/drive/MyDrive/unknown.zip"
extract_folder = "/content/unknown"

os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Extracted files:", os.listdir(extract_folder))


CUDA Available: True
CUDA Device Count: 1
CUDA Device Name: Tesla T4
Extracted files: ['known']
Extracted files: ['unknown']


In [4]:
!pip install face_recognition --no-binary :all:

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face_recognition: filename=face_recognition-1.3.0-py2.py3-none-any.whl size=15518 sha256=bd96552b36dcf60dad9fe11df00e968d37d69da6d80caee8a654e8f8d7dd1d2c
  Stored in directory: /root/.cache/pip/wheels/44/cf/bc/ea4cdfe6f25d298e6c4e0c0da7bbcd1efdc98b7e0c12809805
  Created wheel for face_recognition_models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=96ad0fe81edb80e07a7c8d86d607f59c088cd550f01dd3c2b6011e8f6bbb9525
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face_recognition face_recognition_models


In [8]:
import os
import cv2
import face_recognition
from google.colab.patches import cv2_imshow


!pip install cmake
!pip install dlib
!pip install face_recognition

def read_img(path):
    img = cv2.imread(path)
    if img is None:
        print(f"Error: Could not read image at {path}")
        return None
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    (h, w) = img_rgb.shape[:2]
    width = 500
    ratio = width / float(w)
    height = int(h * ratio)
    return cv2.resize(img_rgb, (width, height))


known_dir = "/content/known/known"
unknown_dir = "/content/unknown/unknown"


known_encodings = []
known_names = []

print("Loading known faces...")


for file in os.listdir(known_dir):
    file_path = os.path.join(known_dir, file)
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        img = read_img(file_path)
        if img is not None:

            face_locations = face_recognition.face_locations(img, model="hog")

            encodings = face_recognition.face_encodings(img, face_locations)
            if encodings:  # Ensure at least one face was found
                known_encodings.append(encodings[0])
                known_names.append(file.split('.')[0])

print("Known faces loaded!")


print("Processing unknown faces...")

for file in os.listdir(unknown_dir):
    file_path = os.path.join(unknown_dir, file)
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        img = read_img(file_path)
        if img is not None:

            face_locations = face_recognition.face_locations(img, model="hog")

            encodings = face_recognition.face_encodings(img, face_locations)

            for face_encoding, face_location in zip(encodings, face_locations):
                matches = face_recognition.compare_faces(known_encodings, face_encoding)
                name = "Unknown"


                face_distances = face_recognition.face_distance(known_encodings, face_encoding)
                best_match_index = face_distances.argmin() if len(face_distances) > 0 else -1

                if best_match_index != -1 and matches[best_match_index]:
                    name = known_names[best_match_index]

                print(f"Match found: {name} for {file}")


                top, right, bottom, left = face_location
                cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.putText(img, name, (left + 2, bottom + 20), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)


            cv2_imshow(img)

print("Face recognition process completed!")


Loading known faces...


RuntimeError: Error while calling cudaOccupancyMaxPotentialBlockSize(&num_blocks,&num_threads,K) in file /root/.cache/uv/sdists-v7/pypi/dlib/19.24.2/E9Wx9I3vvzl2i4p_gKg0d/src/dlib/cuda/cuda_utils.h:186. code: 222, reason: the provided PTX was compiled with an unsupported toolchain.